In [14]:
from platform import python_version

print(python_version())

3.12.2


In [15]:
# Importing the required libraries
import time
from functools import cmp_to_key

# Changing the recursion depth, since this is required for testing
import sys
sys.setrecursionlimit(100000)

In [16]:
# Standard construction of the suffix array
def construct_suffix_array(text, sorting_func):
    # Generate all suffixes
    suffixes = [text[i:] for i in range(len(text))]
    
    # Sort the suffixes using ternary split quicksort
    sorted_suffixes = sorting_func(suffixes)

    # Construct the suffix array
    suffix_array = [len(text) - len(suffix) for suffix in sorted_suffixes]

    return suffix_array

In [17]:
# Custom construction of the suffix array
def construct_suffix_array_custom(text, sorting_algorithm="insertion"):
    # Generate all suffixes
    suffixes = [text[i:] for i in range(len(text))]

    
    # Sort suffixes using bucket sort
    sorted_suffixes = bucket_sort_suffixes(suffixes, sorting_algorithm)

    # Construct the suffix array
    suffix_array = [len(text) - len(suffix) for suffix in sorted_suffixes]

    return suffix_array


# Different sorting methods

## Prefix Doubling

In [18]:
def prefix_doubling(suffixes):
    suffix_to_index = {s: i for i, s in enumerate(suffixes)}
    # First round: Sort them by their first character
    sorted_suffixes = sorted(suffixes, key=lambda x: x[0])
    
    # Assign ranks
    ranks = {}
    rank = 0
    
    for s in sorted_suffixes:
        first_char = s[0]
        if first_char not in ranks:
            ranks[first_char] = rank
            rank += 1  # Increase rank only when a new first character is encountered
    
    # Map each string to its rank
    ranked_strings = {s: ranks[s[0]] for s in sorted_suffixes}
    R = [ranked_strings[s] for s in suffixes]

    is_distinct_ranks = False
    doubling = 1
    
    while not is_distinct_ranks:
        tuple_ranks = {}
        for i, s in enumerate(suffixes):
            try:
                secondary_tuple_rank = R[i + doubling]
            except:
                secondary_tuple_rank = -1
            tuple_ranks[s] = (R[i], secondary_tuple_rank)
        sorted_suffixes = {k: v for k, v in sorted(tuple_ranks.items(), key=lambda item: item[1])}
        
        # Assign ranks
        ranks = {}
        rank = 0
        
        for suffix, tuple_val in sorted_suffixes.items():
            if tuple_val not in ranks:
                ranks[tuple_val] = rank
                rank += 1  # Increase rank only when a new first character is encountered

        # Map each string to its rank
        ranked_strings = {suffix: ranks[tuple_val] for suffix, tuple_val in sorted_suffixes.items()}
        R = [ranked_strings[s] for s in suffixes]
        if len(R) == len(set(R)):
            break
        doubling *= 2
    return sorted_suffixes.keys()

In [19]:
# Test prefix_doubling()
text = 'mississippi'
suffixes = [text[i:] for i in range(len(text))]
prefix_doubling(suffixes)

dict_keys(['i', 'ippi', 'issippi', 'ississippi', 'mississippi', 'pi', 'ppi', 'sippi', 'sissippi', 'ssippi', 'ssissippi'])

## Ternary Split Quick Sort

In [20]:
def ternary_split_quicksort(suffixes, depth=0):
    if len(suffixes) <= 1:
        return suffixes

    # Determine the pivot character at the current depth
    pivot = suffixes[len(suffixes) // 2][depth] if depth < len(suffixes[len(suffixes) // 2]) else ''

    less = []
    equal = []
    greater = []

    for suffix in suffixes:
        if depth < len(suffix):
            char = suffix[depth]
            if char < pivot:
                less.append(suffix)
            elif char > pivot:
                greater.append(suffix)
            else:
                equal.append(suffix)
        else:
            # If depth exceeds the length of the suffix, consider it as 'less'
            less.append(suffix)

    # Recursively sort the partitions
    less_sorted = ternary_split_quicksort(less, depth)
    equal_sorted = ternary_split_quicksort(equal, depth + 1)
    greater_sorted = ternary_split_quicksort(greater, depth)

    return less_sorted + equal_sorted + greater_sorted

## Bucket Sort

In [21]:
def bucket_sort_suffixes(suffixes, sorting_algorithm="insertion"):
    if len(suffixes) == 0:
        return suffixes
    
    # Determine the number of buckets
    num_buckets = len(suffixes)
    max_suffix = max(suffixes)
    min_suffix = min(suffixes)

    # Create buckets and distribute the elements
    buckets = [[] for _ in range(num_buckets)]
    for suffix in suffixes:
        index = int((ord(suffix[0]) - ord(min_suffix[0])) / (ord(max_suffix[0]) - ord(min_suffix[0]) + 1) * num_buckets)
        buckets[index].append(suffix)

    # Sort each bucket using the specified sorting algorithm
    if sorting_algorithm == "insertion":
        for i in range(num_buckets):
            buckets[i] = insertion_sort_suffixes(buckets[i])
    elif sorting_algorithm == "selection":
        for i in range(num_buckets):
            buckets[i] = selection_sort_suffixes(buckets[i])
    else:
        raise ValueError("Invalid sorting algorithm or not implemented yet")

    sorted_suffixes = []
    for bucket in buckets:
        sorted_suffixes.extend(bucket)

    return sorted_suffixes

### Insertion and Selection sort (for the use in bucket sort)

In [22]:
def insertion_sort_suffixes(arr):
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
    return arr



def selection_sort_suffixes(arr):
    for i in range(len(arr)):
        min_idx = i
        for j in range(i + 1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

## Naive implementation (built in)

In [23]:
def cmp_suffix(i,j):
    if (i < j):
        # Suffix i is lexicographically less than suffix j
        return -1
    if (i == j):
        # Suffix i and j are equal
        return 0
    else:
        # Suffix i is lexicographically greater than suffix j
        return 1
    
def suffixes(s):
    arr = []
    for i in range(len(s)):
        suffix = s[i:]
        arr.append(suffix)
    return arr

# Testing (To check if it works)

In [24]:
# Testing 
text = "banana"
suffix_array_builtin = sorted(suffixes(text), key = cmp_to_key(cmp_suffix))
print("Suffix array using built in sorting function:", suffix_array_builtin)

suffix_array_prefix_doubling = construct_suffix_array(text, prefix_doubling)
print("Suffix array using prefix doubling:", suffix_array_prefix_doubling)

suffix_array_quick_sort = construct_suffix_array(text, ternary_split_quicksort)
print("Suffix array using ternary split quicksort:", suffix_array_quick_sort)

suffix_array_insertion = construct_suffix_array_custom(text, sorting_algorithm="insertion")
print("Suffix array using insertion sort:", suffix_array_insertion)

suffix_array_selection = construct_suffix_array_custom(text, sorting_algorithm="selection")
print("Suffix array using selection sort:", suffix_array_selection)

Suffix array using built in sorting function: ['a', 'ana', 'anana', 'banana', 'na', 'nana']
Suffix array using prefix doubling: [5, 3, 1, 0, 4, 2]
Suffix array using ternary split quicksort: [5, 3, 1, 0, 4, 2]
Suffix array using insertion sort: [5, 3, 1, 0, 4, 2]
Suffix array using selection sort: [5, 3, 1, 0, 4, 2]


# Testing running time with random genetics string

In [25]:
######################################
#### REAL TESTING FOR REAL MEN #######
#### With random genetics string #####
######################################
x = 'gactaagttaacacacgcatcggccgttccgacatccaactggttcctccatcggggccaatttccattctactgagggattccttgaggatattgcgagcgcgcgtggacattagcggtgtgtctgaaccacgtcttagacgtcaatatctgcgcaatcaagaaatgatcgtccaactgaggatgcaccatcttgtataacgcaatagacggactcatggtaagtgtcagcgtagtatcacagaactatcagagatctaaatcctcagttccttgtgagttggctaagaccagcgccggtactccggggggccgtatcttaggtgtaaccccgaagttcgccccagcttaagcagccacttagatgagggcctcgctgccagacgtcctcgctgggggccaatttataaccgactcagttacatccgcggggagttcgttgaaacaccggaggccgctgggtagtctttgtcgtacgtactttcggagcttccattcgggtgctcgacacagcgcgcttaaactcgctgttcataacacctggatatacgtgcgtcgagctgctattcttccatatacgatagggctagacacatcgcttaagtggccatcccttagacttctttacctgtgcgttagcttcattcggtttgcaaacgcaggcagtttgagccgtgaactggacgagaacgcgtggtgtctggcatattctttccggttcggaatagagggtcttttgatcgattcctgggccgtgtggctcgaaactttctatggagccgccagcagtaaagatgcatttacttgtgatatgggcaatccgcttctcgttggatccaggcaataaaaaagttccctcccgattcagtattcggactgagataggccatgaggtgatgttcagacttctaggatcgcggacgctgacccgcatattcgaccacggagacggcgtaatccgtgaatgtccttatttcgcactgttacacgtgctgagaggtgttgccgccaaaaaccgctgggttcaacagccggcaccggtttcgacatcgtttttcagcggcctcactaattattgtacccgagcaggaacagattgcagcgggctgccacgcgaaccaacctggagggtggtgtgggcggttcaatgacgtagcctctatcaagactgaaaggaaagtatgtacgatctctagccagacacgccgttatacttgagttaaggcttcaccctctcgagaacccgcgatgattggtatcgcgccgttcaattctctataaagattcttattcacagccccaggccgtggtacgtcaagcggatgcggaataaaaccacacaaaggattttccggggtccgagttcgtatcacgtatggtagaggttagaaatattttgtgaattgcaattctggcgataatcgttgcagtctgatggcggcgtgattagggtcggcatccctcgcagaaatgggagggcgcctcctgaccgtctaacggttatctgaaacggatgttagcgcgaagcatctaatatatcttcgggtacgactggaggtcaagtgggccgactggcggctttaactgcttaggcatgtatagccattgccaagaaatgccaggctgccaggggaggacctcagagagcggaccgtttgttatcccagagaggaggggacagggacacttctccaaattcgtccggttcataaagaccacttttatacatatggctctatcggttaggtgagggaggctgatactgtttgcgatcgtacatctgacctgtgagttcccgttaggaattatccgcttgtagattttccggacagacacaaaatgtataatgtccgtactccatagtaaaaccctatcattacttcatggagccgggcccgagtttccctgcgagaagccttgacctcctgagattagcgatccatcctaatatgagatcccaagcctgacatatggaccagattcggtcactgactctgatgttcacatctaatgagtaggttacgccccgagacacaactctaacaggacccacctaaacgtcgtacggttggacctggttcatacagctagtgtctctcaagggctcatggacaaccccgcaggataaagtatgtgtccaatttatcgcacggtcgttcttttcaataaagcccattagatctgaattgtttctgactttatcatagtgcaagtggatgatgacgagtggggccctcgacgataatagatgcctgctactcatacgccggtggaggcgagtgaactgacccggtcgcgtataggctgcggtagaggtttcttgaatgtagccacaaatgcaatgacatcatctctctagatcacgtcagctgaatacggaagtcgatgaaccgccaaagattcctacttcaaatcgtagcctatttttagttccttgggtaagacagtcgtgaccgaaagcaggtatatgcatctggtatccatttgccacgcacctagcaccccgctatagtttaatcgctcaattaccctcctagaatcgaagtctgagaaggctaggttagtaaattggccgctgtaggcggtgcatgcgcacaggcatttgcctatcgctaactgaggtccagacagttatacagcagactcataataaccgcacccaccgcagtagccctatctatctaccccattcttttcgttgaaggttccgattctgccttgtagccggagacgacatggcttcctgacttgctataataacgtcgccgagggacatgacccgggactctgagggctcaatacttgtgagtcgtccgccagtacggacatgaggattctacaaatcctgataaaagatgtacgcgtcgatgccccccgctaagcgcatagtgatctgtactctaaccaagtaaactgtgcagcatcgtgatcaccccggaaactgcgttccgcgtatgtgcctaaaggcttcgcagagatatcgccatcgctttccggcgggggtctccggccttaacggggcttcagaaaaatgcaatttctctatcgttgaacgccgggtgttccaaagaattcggaatcaaagcctgcgaggacaagggtttacctataaagccaccaggactaatcgattggcgtgacgctgaaaagttcgaaggtgcaggcttcgtctcgcattaggtgtcatcaagtggaattaaaaggcggggcgggcgtcaaagtcgttgggctcctacatactgcaagtattacggtggttggggtaagcctcactatacggcctgtcggccgcctcttactaggcattccttccttaaagtacaccttattcgagcacacacacccgtactacacccgaggtctgtctgtacagacatggtgccatttgctggtgtaatacggcccactacatagggacaaggcatcctccgcgagtctaccaaatactgcgatttctatcctatggaatttcggacggtcgatgggcggaacaagaaggaagatgcctacacaacatcaataggtccaattcaattgctctctcatcgctatgagtgtggttaaaggtctcctatttagaattagaaggcacgggcagtatttccctctgcgatttcgtcacgcttaaagtcatcccgatatgttgtacgagtctagagtaatccgcgcattgtaatgcttacacttgagtcagaagggaggtgtactaggtcatatttctccacgattgttccccgtgaaggcgtagttaccaaatatgtaatagataatgtcatagggggactgtcctctaagtaccctcttctaacgcataaccatgagtgattctggccacaagatatctctgcgaggtcagtcagtaattccatcaattacggtgtgcaacaagagacagacactaatagtttaggtctactatgtcgggagggatttctttggagcctgtgggcaattcgtgaaatggacctaactaagcttacttgcagaggcattgttcctgagctccccttaattaattcaaaccagagatgacagttgtacttaacgtagaatcagcagttgaaggatacaatcttttggaagtagttactagttgccacgatgcagtctgaagtttattgagcgaacggacactagcggatatgtataagacgtccaccgtcgcagcttcgcacttattaattaaggaggcgcttgacattttaacgagtattctctaccctaaaactctgttagaatcgcgagcctaagcaaaaagcggcgggatctatcgaaggtaatacactcataatgaagtagtccgggtgcgggggacacaggccacatcctaccgagccatacctgtgcatcatagtagacagagtgactgataagtgatcagtcattagtttgccactacttgtcttactcgacccaggagcgaatactcggagctcggtagcgctctttcatgattttggacggcacatggcgagttaagctaggatcatgcttcgtaccacccccgctatgagatctcgtcaattgggactactgtctttagtcagctcggataactaaaagtggggacaatgccggatgtggcttaaggctacgatcgtatccagggatgtcaggtctcctgtcataatgcgatcagagtacagtctcatgctaacgagggacgggaaccaagttcaatgctggcgtattggcctcacccccagaggtgtcccaggatgtgtatctataaatttcagactcactcgaaagaacttgatactcttgccgtggcggtcaagctttgcgatcctactcccctgtgataatgcctacgtccgcctaagctgctggtgcgatggactcccgacggctcggtactactgatagcgctatatcgacagtgttaataagtctgagccccttcatagataattccaggtcaccagcgagaacgatagcggccgagtccccatgacatcaacgtgggctcctctgtgcctgcgaagtaagcctagtatgctgtcggtgtattacgaaccgatttggagctcagttaagttccctagtgtgatttcacttagcaccttcgaacgctctccacaccaattatggtagttgtttcagtagtttcactagattccatgagggatctgcggtcagtagcaggatcggaaatcacccgtcatgtttaagcctataaggacgttcagcggtttgggatgcttaatgaggttatacgacacatgattcctggtctgccgtagtgtacaacataagtaacttgttacgctctccaccttttagaatattgtcaggccccaacgggataacggaacgtggaacagatccgtagcaaagaggcaatcttccgcgaatgtgaagcgaacccactcgcatgatacaattctatgggcataccattcccaaacattaagtataacgctggatttaggggaacaaggtctgacactgagtgaacagcgccagacaattaaaacggatgaaacgacaattaactagtgctaacagcagatatagattagggatactctttttgtgccacccatcgtaatatccaaaactctccggttcaagccctttcaactactttccgctcccgccgtgccaaaacaacagggcggtgattacgaacatgattatgttccgacgcttcgaagaactcaaagttagctgtagctagttcgcccagtcccgaaacagtcggcccttgagctcgagcgaccagtatgtggcaaaattccgaactatagtactactctgtcgttaacattttcctataaatcgtgaagcttagctccccttttagcggccgtaacgagaataatagttcaagaaataccgcattcgcgtgtcatctggttcgcgccctcgcgtgttcgacagccagacacaaatactccagtagggagcgacgtcagagtcggatcccagagtcatctcgtgaagctgtcagtgctggacaggtacagtcaagacaattaaatcgtctagagcctacccagattggagtatactggtgtgccaataagggcttgcctacaccctcgcgagaacagagtgtgtaacttcgaaatgccgtcgagcccttataaaaggattgtctaggattttccgagtctcgatctggccgattgcaaagtgacgcctgtatgaggcgcgttatacactaacatgcctgagataaagaatacttcgggatgccgcgcgtctgatggtaaggtagcgatcgggaggcctaccggaccgaatgaaccgtatgaataactaaatccccatctgattcaacggtaaactcattatgcagccgagggggcgtcacattttttcgataaactcgtacgtggctccgtggaagctgtctcaggatcctggtagaatctcccccctcctcttaccctaaaccgggagatggcccctataataagggactatgtaactatcaagagtttgaagtggtacccaaactcatgcccatcccttgtgtacaactaagggtattacacatgcatatgccactagaaatacgtcgatgattcgtcgagtagagtttcttgccttgggaacttgcgcttggattgtctgtcacgcgggacgtgtctttgcccatgacaggtctgcgtgacttagatacgatcttctagcggttgcacgtgtaccgtaatttgatatacgtggccacatacgttcgtaactcatgtctggacgggattataaaagggtaggacactatctcgatccacgggtagtgcagcaacacgatggtcattaaggatggcccggacgacgttgcaaaggcgccacaccacgctcgctcttaactagagatcggcagctgagaccgcgtacaggatattattaagcctgactgagacaaagcagcgatggacctagcggtgatttgacccaagcatatagcgcatgtttacagacgccaccacccatactattacaaggtctcctcactttgaaacccaacacgccctttctagacatccagagtcgattccaaacggtaaaaagggaggggaaccttaacagctacgaccaactgcgatgagcgactccagaaagcatcagttccccacgatattgaatttgagtatatgttcccatggacaggttacaatagaccttactaacaagcgctttatatcacctcagcccatttcttctctccctaccgataggctcccgtctatggtccgccagtgccttgttcgaagtaggtaccgctatttccgcccggcgagtattactcaggaacctaccttacagatgtcacggtgccatcaagccacagcatggcccaagagcttttgtcgtcaggccgtgactttacgctactccctttatgagcgcatatactcgagaatgtcgcatccgctcgtctggataccgaggttccgtgattctctttattgcaagggcagagaatgtggcatgatcgcccctgatcgacatagcctccccattcggtacttcaccgctctacgtaatctcgtgcatgactcaggcggaaattgacgagcacaaccagagtgtatcctggagattcacggactatgcgcggctgttataggatgagtacgcaccttcaaccagtcgaatgttaaaccggttgcgacagccctgtctagcgggttataaaagcttgttggatacggaacccccctcaggtatattaccccccaaagatacgcagtttcgaattacggtcggtgatctaattccgcaccccggtgcgtagggcgcagcggcagatcccaaatctcttcgtgaggccacagattataggctaacttgctctataatcctgcccgctgtctctagaccaaacgatgacttatttagcgagtccgcactagcagggccctcgctttccgcaaaaccacggcgcacgtaaagggtagtcgggagccaggcgagtctctaccacatctgaatggagctaggggtagccgacgttgtcgactttcacaaaatcgcacgtttctcttgacagtatttggtctcgcttgcggcttggatattaccctgcatctatgagacctgcgaataacgcgggaaatgtgcccacaacgcccaacgttctctacaccctctcggcaaccgcccgactgggagtcatcagcttgtagagtcgatgagttatcggactacataggccttggcacattttaaaattgagggttagttcaagtcccgcaaccatcttagaccacgtcactattatgcgtagcgagcatacgcggcaatgatcccgtcagtaacagtcgtgaataacgcagcggaatcacgagcggcatcgccatctatgtactatgagtggaaaacggtgcggcacgtatttccctaggtatatacttagtcaaccgctatggtttacgtaaatggggttgcgcaggacatcgtaagatgactaaccatcccaccggcatatggagcctttgtttaaagcatgttggttgtcgtatggcagaactctccccgccgtagcccctgatatgcaacgcatgattgactacaacataagtcagaccacctgacatcaaacagcgcccaacgattcttgtgaacaatcccctgttcccacaaagggaagcaccgcgcagggatgcataaccaatacgctgtccggatagacttctacgtttgctacggggccttttaggttatagccattattactgtgatgtgcggaaggtgcagtagaagttggtcatagacttagattggagagtacatcgaccgaaagacttcaaccagtgtgcaggtagttttagtcgacgtaagttagttcagagacttcaatcaccttaacatagaccatataagagattaggttcttcaaattgccttcggtgattgaggcgtcacggcctgatcgatgtgacccgttgcttaaccgccgacactcaattcaccactcgtctcggactcttctgaaacgcgtacctgcgacggacccatgctcatgaaacacacaaatccggttatgtttttctccttcctaactagtatcacctgctaaaactaaacgtatcattccgccgtgcgattaagaagctgaaatgaagtggcgtaacgagtgaataacttctttaaagtaaggagcctgggtcgagatcaggaccacgatgcgggtccagacgcagccaggatcttgcgacgtataacgcctttaagtctaggctgttggactccacttccatcctgtagacgcgcattcttggattgaagctggaggtcataactcgtattgaaggctcttgccccctcttgccgcacagaagaaagggccctagtatttggattagaattgttgcgttaatacactcgtgaaaactgcgatagctcctctattagtcggcggcgcgcgaaaaaaactcgttacggcttcctcagagacttcggacacacctcagccaccggtttgttcgtgcacgtgacacagggtcccgctgggttcgagatcctattacatacataacttattatgccctaccttaggccgctcagtaggctgggcttaccgcaataacccttgttagtgctctaggaagggtatatagcgagaactgtagactcctgctcggtattgtaagtcgggaaacttgtcgaaggcagtatccggattttagtgggcctcggcccgtgtttcccatttcaactaaagacggagccggagctaacgtagccgcgtgcatcctttattacgtgggagcatcccgatgcgatctatagtagtcggatagaatcgtaagattctagtcaccatgcgtggagattaggatagcccacaaccccggcactgattcaaatgtcaaaagatttcgtttccattgtaggcttggattcacgaccgccctaaacatttagcattcagggccggggcgccgacaacaatcgttagtcagaagggaacctgggcctacgactccaggttcgagaccgggtccttgggaagagcgttttcctgtggaaaatagaaatgatc'
start_time_builtin = time.time()
suffix_array_builtin = sorted(suffixes(x), key = cmp_to_key(cmp_suffix))
end_time_builtin = time.time()
running_time_builtin = end_time_builtin - start_time_builtin
print(f'Running time with string is {round(running_time_builtin, 4)} seconds using built in sorting function')

start_time_prefix_doubling = time.time()
suffix_array_prefix_doubling = construct_suffix_array(x, prefix_doubling)
end_time_prefix_doubling = time.time()
running_time_prefix_doubling = end_time_prefix_doubling - start_time_prefix_doubling
print(f'Running time with string is {round(running_time_prefix_doubling, 4)} seconds using prefix doubling')

start_time_quick_sort = time.time()
suffix_array_quick_sort = construct_suffix_array(x, ternary_split_quicksort)
end_time_quick_sort = time.time()
running_time_quick_sort = end_time_quick_sort - start_time_quick_sort
print(f'Running time with string is {round(running_time_quick_sort, 4)} seconds using ternary split quicksort')

start_time_insertion = time.time()
suffix_array_insertion = construct_suffix_array_custom(x, sorting_algorithm="insertion")
end_time_insertion = time.time()
running_time_insertion = end_time_insertion - start_time_insertion
print(f'Running time with string is {round(running_time_insertion, 4)} seconds using insertion sort')

start_time_selection = time.time()
suffix_array_selection = construct_suffix_array_custom(x, sorting_algorithm="selection")
end_time_selection = time.time()
running_time_selection = end_time_selection - start_time_selection
print(f'Running time with string is {round(running_time_selection, 4)} seconds using selection sort')


Running time with string is 0.056 seconds using built in sorting function
Running time with string is 0.134 seconds using prefix doubling
Running time with string is 0.08 seconds using ternary split quicksort
Running time with string is 1.0767 seconds using insertion sort
Running time with string is 1.4045 seconds using selection sort


# Testing (With a string of a's)

In [26]:
######################################
#### REAL TESTING FOR REAL MEN #######
#### With a string of a's      #######
######################################
x = 'a'*10000
start_time_builtin = time.time()
suffix_array_builtin = sorted(suffixes(x), key = cmp_to_key(cmp_suffix))
end_time_builtin = time.time()
running_time_builtin = end_time_builtin - start_time_builtin
print(f'Running time with string is {round(running_time_builtin, 4)} seconds using built in sorting function')

start_time_prefix_doubling = time.time()
suffix_array_prefix_doubling = construct_suffix_array(x, prefix_doubling)
end_time_prefix_doubling = time.time()
running_time_prefix_doubling = end_time_prefix_doubling - start_time_prefix_doubling
print(f'Running time with string is {round(running_time_prefix_doubling, 4)} seconds using prefix doubling')

start_time_quick_sort = time.time()
suffix_array_quick_sort = construct_suffix_array(x, ternary_split_quicksort)
end_time_quick_sort = time.time()
running_time_quick_sort = end_time_quick_sort - start_time_quick_sort
print(f'Running time with string is {round(running_time_quick_sort, 4)} seconds using ternary split quicksort')

start_time_insertion = time.time()
suffix_array_insertion = construct_suffix_array_custom(x, sorting_algorithm="insertion")
end_time_insertion = time.time()
running_time_insertion = end_time_insertion - start_time_insertion
print(f'Running time with string is {round(running_time_insertion, 4)} seconds using insertion sort')

start_time_selection = time.time()
suffix_array_selection = construct_suffix_array_custom(x, sorting_algorithm="selection")
end_time_selection = time.time()
running_time_selection = end_time_selection - start_time_selection
print(f'Running time with string is {round(running_time_selection, 4)} seconds using selection sort')


Running time with string is 0.0596 seconds using built in sorting function
Running time with string is 0.2701 seconds using prefix doubling
Running time with string is 11.751 seconds using ternary split quicksort
Running time with string is 26.1508 seconds using insertion sort
Running time with string is 29.8165 seconds using selection sort
